In [24]:
import torch
from facenet_pytorch import InceptionResnetV1

# Load the pre-trained FaceNet model from facenet_pytorch
model = InceptionResnetV1(pretrained='vggface2').eval()

# Create a dummy input tensor with the shape the model expects
dummy_input = torch.randn(1, 3, 160, 160, device='cpu')

# Export the model to an ONNX file
onnx_model_path = 'facenet.onnx'
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True, input_names=['input'], output_names=['output'])


In [25]:
import onnx
from onnx import checker

model = onnx.load('facenet.onnx')
# Check the model
try:
    checker.check_model(model)
    print("The model is consistent and valid.")
except Exception as e:
    print("The model is invalid: ", e)


The model is consistent and valid.


In [23]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model_path = 'facenet.onnx'
onnx_model = onnx.load(onnx_model_path)

# Convert the ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model
tf_model_path = 'facenet_tf'
tf_rep.export_graph(tf_model_path)


ModuleNotFoundError: No module named 'tensorflow_addons'

In [6]:
import cv2
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='facenet_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_image(img):
    img = img.convert("RGB")  # Ensure image is in RGB format
    img_resized = img.resize((160, 160))
    img_array = np.array(img_resized).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Load and preprocess the single input image
input_image_path = "image.png"  # Change this to the path of your input image
input_img = Image.open(input_image_path)
input_face_encoding = preprocess_image(input_img)

def get_face_embedding(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index']).flatten()

input_face_encoding = get_face_embedding(input_face_encoding)

def match_face(face_encoding, input_face_encoding, threshold=0.6):
    similarity = cosine_similarity([face_encoding], [input_face_encoding])[0][0]
    return similarity, similarity > threshold

input_video_path = "video.mp4"  # Change this to the path of your input video
output_video_path = "outputi.mp4"

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Set output FPS to match input video

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0
batch_size = 32
frames = []
skip_frames = 0  # Initialize skip_frames counter
skipped_frames = []  # To store skipped frames
min_skip_frames = 2  # Minimum number of frames to skip
max_skip_frames = 10  # Maximum number of frames to skip
adaptive_skip = min_skip_frames  # Start with minimum skipping

def process_frame(frame, frame_count):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) > 0:
        matches_found = False
        print(f"Faces found in frame {frame_count}: {len(faces)}")
        for (x, y, w, h) in faces:
            face_image = frame[y:y+h, x:x+w]
            face_pil = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)).resize((160, 160))
            face_encoding = preprocess_image(face_pil)
            similarity, is_match = match_face(get_face_embedding(face_encoding), input_face_encoding)
            print(f"Frame {frame_count}: Cosine Similarity = {similarity}")
            if is_match:
                print(f"Match found in frame {frame_count} (Similarity: {similarity:.2f})")
                return frame
    return None

def process_skipped_frames(skipped_frames):
    if skipped_frames:
        frame, frame_count = skipped_frames[-1]
        processed_frame = process_frame(frame, frame_count)
        if processed_frame is not None:
            out.write(processed_frame)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if skip_frames > 0:
        skipped_frames.append((frame, frame_count))
        skip_frames -= 1
        continue

    frames.append((frame, frame_count))

    if len(frames) == batch_size:
        with ThreadPoolExecutor(max_workers=mp.cpu_count()) as executor:
            results = list(executor.map(lambda f: process_frame(f[0], f[1]), frames))
            faces_detected = False
            for result in results:
                if result is not None:
                    out.write(result)
                    faces_detected = True
                    process_skipped_frames(skipped_frames)
                    skipped_frames.clear()
            if not faces_detected:
                adaptive_skip = min(adaptive_skip + 1, max_skip_frames)
                skip_frames = adaptive_skip  # Set to skip the next frames adaptively
            else:
                adaptive_skip = min_skip_frames  # Reset to minimum skipping if faces are found
        frames = []

# Process remaining frames
if frames:
    with ThreadPoolExecutor(max_workers=mp.cpu_count()) as executor:
        results = list(executor.map(lambda f: process_frame(f[0], f[1]), frames))
        for result in results:
            if result is not None:
                out.write(result)

cap.release()
out.release()
print(f"Trimmed video saved as {output_video_path}")


ValueError: Could not open 'facenet_model.tflite'.

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pyautogui
import time

def search_youtube_and_download(start_episode, end_episode):
    s = Service(r'C:\Users\User\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=s)
    driver.get("https://www.youtube.com")
    
    for episode in range(start_episode, end_episode + 1):
        search_query = f"tmkoc episode {episode}"
        search_box = driver.find_element_by_name("search_query")
        search_box.clear()
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)  # Wait for page to load results
        
        # Click on the first result
        try:
            videos = driver.find_elements_by_id('video-title')
            if videos:
                videos[0].click()
                time.sleep(10)  # Wait for video to load
                
                # Here you would continue with IDM or download interaction
                # Make sure to adjust PyAutoGUI coordinates as per your IDM settings

            else:
                print(f"No results found for {search_query}")
        except IndexError:
            print(f"No results found for {search_query}")

    driver.quit()

search_youtube_and_download(3700, 3800)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 125
Current browser version is 124.0.6367.209 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF747C022C2+60002]
	(No symbol) [0x00007FF747B7CA59]
	(No symbol) [0x00007FF747A37EDA]
	(No symbol) [0x00007FF747A76242]
	(No symbol) [0x00007FF747A75418]
	(No symbol) [0x00007FF747A6F007]
	(No symbol) [0x00007FF747A6AFA4]
	(No symbol) [0x00007FF747AB79A4]
	(No symbol) [0x00007FF747AB7000]
	(No symbol) [0x00007FF747AAC033]
	(No symbol) [0x00007FF747A79657]
	(No symbol) [0x00007FF747A7A251]
	GetHandleVerifier [0x00007FF747F13E2D+3278285]
	GetHandleVerifier [0x00007FF747F60190+3590448]
	GetHandleVerifier [0x00007FF747F561D0+3549552]
	GetHandleVerifier [0x00007FF747CB1DE6+779654]
	(No symbol) [0x00007FF747B87ACF]
	(No symbol) [0x00007FF747B82EE4]
	(No symbol) [0x00007FF747B83072]
	(No symbol) [0x00007FF747B72C4F]
	BaseThr

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Path to your Chromedriver
chromedriver_path = r'C:\Users\User\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Specify the location of Brave
brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

# Path to the extension
extension_path = r'C:\Users\User\Downloads\IDMGCExt\IDMGCExt.crx'

# Set up Selenium options to use Brave with the extension
options = Options()
options.binary_location = brave_path
options.add_extension(extension_path)

# Set up the Chrome service
service = Service(executable_path=chromedriver_path)

# Create a new instance of the Brave browser with the extension loaded
driver = webdriver.Chrome(service=service, options=options)

# Open a website, for example, Google
driver.get("https://www.google.com")

# Keep the browser open for a while to observe the extension in action
time.sleep(20)

# Close the browser
driver.quit()


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pyautogui

# Setup
chromedriver_path = r'C:\Users\User\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"
options = Options()
options.binary_location = brave_path
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # Navigate to the IDM extension on the Chrome Web Store
    driver.get("https://chrome.google.com/webstore/detail/idm-integration-module/ngpampappnmepgilojfohadhhmbhlaek")
    time.sleep(5)  # Allow the page to load

    # Click the 'Add to Brave' button using XPath to find by text content
    add_to_brave = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Add to Brave')]")))
    add_to_brave.click()

    # Use pyautogui to handle the "Add Extension" dialog
    time.sleep(2)  # Allow the dialog to appear
    pyautogui.moveTo(747, 55)  # Coordinates for the "Add Extension" button
    pyautogui.click()
    input("Check if the extension was added and press Enter to continue...")

    # Navigate to YouTube and search for the video
    driver.get("https://www.youtube.com")
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "search_query")))
    search_box.send_keys("Taarak Mehta Ka Ooltah Chashmah episode 3750")
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)  # Wait for results

    # Click on the video to play it
    video = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "video-title")))
    
    video.click()
    time.sleep(10)  # Wait for the video to start playing

    pyautogui.click(728, 65)  # X, Y coordinates for IDM download button
    time.sleep(2)
    # Use pyautogui to handle IDM download
    pyautogui.click(784, 113)  # Coordinates for IDM download button

    input("Press Enter after you start the download manually...")

finally:
    driver.quit()


##TEST

In [3]:
from facenet_pytorch import InceptionResnetV1, MTCNN
from moviepy.editor import VideoFileClip, ImageSequenceClip
import torch
import numpy as np
import torchvision.transforms as transforms

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load MTCNN for face detection
mtcnn = MTCNN(keep_all=True, device=device)

# Define paths
input_video_path = "jeta.mp4"  # Replace with your video file path
output_video_path = "output_test.mp4"

# Initialize the video clip and get properties
video_clip = VideoFileClip(input_video_path)
fps = video_clip.fps

# Transformation to convert numpy frames to tensors
transform = transforms.ToTensor()

# Collect frames with faces
frames_with_faces = []
duration = 5  # Limit capture to 5 seconds
end_time = min(duration, video_clip.duration)

for t in np.arange(0, end_time, 1.0 / fps):
    # Get frame at time `t`
    frame = video_clip.get_frame(t)
    
    # Convert frame to tensor for MTCNN processing
    frame_tensor = transform(frame).to(device)
    
    # Detect faces in the frame
    boxes, _ = mtcnn.detect(frame_tensor.permute(1, 2, 0).unsqueeze(0))
    
    if boxes is not None:
        for box in boxes:
            if box is not None:  # Ensure box is not None
                box = [int(b) for b in box]
                # Draw a rectangle (or some other marking) around the detected face
                frame[box[1]:box[3], box[0]:box[2]] = [0, 255, 0]  # Highlight the face region in green

        # Add frame with faces to the list
        frames_with_faces.append(frame)

# Save the detected face frames to a new video file
output_clip = ImageSequenceClip([frame for frame in frames_with_faces], fps=fps)
output_clip.write_videofile(output_video_path, codec="libx264")

print(f"Test video saved as {output_video_path}")


Using device: cuda
Moviepy - Building video output_test.mp4.
Moviepy - Writing video output_test.mp4



Moviepy - Done !
Moviepy - video ready output_test.mp4
Test video saved as output_test.mp4


In [3]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load pre-trained FaceNet model for feature extraction
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Initialize OpenCV DNN face detector
dnn_face_detector = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

# Define a consistent preprocessing pipeline
face_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Normalize embeddings
def normalize_embeddings(embedding):
    return embedding / embedding.norm(dim=1, keepdim=True)

# Preprocess reference images with augmentation
def preprocess_reference_images(folder):
    reference_embeddings = []
    for file in os.listdir(folder):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, file)
            img = Image.open(img_path).convert('RGB')
            for _ in range(10):  # Augment each image 10 times
                augmented_img = transforms.functional.adjust_brightness(img, np.random.uniform(0.8, 1.2))
                augmented_img = transforms.functional.rotate(augmented_img, np.random.uniform(-15, 15))
                img_tensor = face_transform(augmented_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    embedding = facenet(img_tensor)
                reference_embeddings.append(embedding.squeeze(0))
    if reference_embeddings:
        return normalize_embeddings(torch.mean(torch.stack(reference_embeddings), dim=0).unsqueeze(0).to(device))
    else:
        raise ValueError("No reference images found!")

# Load reference embeddings
reference_folder = 'face found'  # Folder containing reference images
reference_embeddings = preprocess_reference_images(reference_folder)

# Global threshold for similarity
FACE_THRESHOLD = 0.85
COMBINED_THRESHOLD = 0.75

# Function to process frames
def process_frame(frame, reference_embeddings, frame_number):
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    dnn_face_detector.setInput(blob)
    detections = dnn_face_detector.forward()

    h, w = frame.shape[:2]
    faces = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype("int")
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
            face_img = frame[y1:y2, x1:x2]
            if face_img.size == 0 or (x2 - x1) < 50 or (y2 - y1) < 50:  # Skip small or invalid faces
                continue
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
            face_img = Image.fromarray(face_img)
            faces.append((face_img, (x1, y1, x2, y2)))

    if faces:
        face_tensors = torch.stack([face_transform(face[0]) for face in faces]).to(device)
        with torch.no_grad():
            face_embeddings = facenet(face_tensors)  # Shape: (num_faces, embedding_dim)
        face_embeddings = normalize_embeddings(face_embeddings)

        # Compute cosine similarity between face embeddings and all reference embeddings
        similarities = torch.mm(face_embeddings, reference_embeddings.T)  # Shape: (num_faces, num_references)

        matches = (similarities > FACE_THRESHOLD).any(dim=1)
        if matches.any():
            matched_indices = torch.nonzero(matches).squeeze(1)
            for idx in matched_indices:
                max_similarity = similarities[idx].max().item()
                print(f"Match found in frame {frame_number}! Face similarity: {max_similarity:.2f}")
                return frame
    return None

# Video processing setup
input_video_path = "brrr.mp4"
output_video_path = "output1.mp4"
cap = cv2.VideoCapture(input_video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS) or 25)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_number = 0
consecutive_detection_mode = False
consecutive_frames_left = 0

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Adaptive frame skipping
adaptive_skip = 1  # Start with no skipping
max_skip = 5       # Maximum number of frames to skip

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1
    processed_frame = process_frame(frame, reference_embeddings, frame_number)

    if processed_frame is not None:
        out.write(processed_frame)
        print(f"Face detected in frame {frame_number}. Entering consecutive detection mode.")
        consecutive_detection_mode, consecutive_frames_left = True, 24
        adaptive_skip = 1  # Reset adaptive skip

    elif consecutive_detection_mode and consecutive_frames_left > 0:
        out.write(frame)
        consecutive_frames_left -= 1
        if consecutive_frames_left == 0:
            consecutive_detection_mode = False
        adaptive_skip = 1  # Reset adaptive skip

    else:
        adaptive_skip = min(max_skip, adaptive_skip + 1)
        for _ in range(adaptive_skip - 1):
            cap.grab()
            frame_number += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Trimmed video saved as {output_video_path}")


Using device: cuda
Match found in frame 115! Face similarity: 0.87
Face detected in frame 115. Entering consecutive detection mode.
Match found in frame 116! Face similarity: 0.91
Face detected in frame 116. Entering consecutive detection mode.
Match found in frame 117! Face similarity: 0.91
Face detected in frame 117. Entering consecutive detection mode.
Match found in frame 118! Face similarity: 0.92
Face detected in frame 118. Entering consecutive detection mode.
Match found in frame 119! Face similarity: 0.91
Face detected in frame 119. Entering consecutive detection mode.
Match found in frame 120! Face similarity: 0.87
Face detected in frame 120. Entering consecutive detection mode.
Match found in frame 121! Face similarity: 0.88
Face detected in frame 121. Entering consecutive detection mode.
Match found in frame 122! Face similarity: 0.91
Face detected in frame 122. Entering consecutive detection mode.
Match found in frame 123! Face similarity: 0.92
Face detected in frame 123. E